In [ ]:
import os
import re
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Flatten, InputLayer
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import utils
import keras
import imageio 
from PIL import Image 

In [ ]:
# Reading the data
train = pd.read_csv('agedetectiontrain/train.csv')

# Image resizing of test data into single numpy array
temp = []
for img_name in train.ID:
    img_path = os.path.join('agedetectiontrain/Train', img_name)
    img = imageio.imread(img_path)
    img = np.array(Image.fromarray(img).resize((32, 32))).astype('float32')    
    temp.append(img)

train_x = np.stack(temp)

# Normalizing the images
train_x = train_x / 255.

# Encoding the categorical variable to numeric
lb = LabelEncoder()
train_y = lb.fit_transform(train.Class)
train_y = keras.utils.to_categorical(train_y)

In [ ]:
# Specifying all the parameters we will be using in our network
input_num_units = (32, 32, 3)
hidden_num_units = 500
output_num_units = 3

epochs = 100
batch_size = 128

In [ ]:
def weight_initializer(list_of_weights, init_weight_names):
    for weight_init in range(len(list_of_weights)):        
        # Writing structure of model  
        model = Sequential([
          InputLayer(input_shape=input_num_units),
          Flatten(),
          Dense(units=hidden_num_units, kernel_initializer=list_of_weights[weight_init], activation='relu'),
          Dense(units=output_num_units, kernel_initializer=list_of_weights[weight_init], activation='softmax'),
        ])
        # Defining parameters like optmizer, loss function and evaluating metric
        model.compile(loss='categorical_crossentropy', # 
                      optimizer=keras.optimizers.Adam(), # Learning rate and momentum can be passed inside optimizer
                      metrics=['accuracy'])

        # Traning the model and writing log file for TensorBoard
        
        logdir = r'weights\\' + init_weight_names[weight_init] # Each log file needs to be written in a distinct directory. (Mandatory)        

        # To store tensorboard graph logs
        cb = keras.callbacks.TensorBoard(log_dir=logdir, write_graph=False) 
        
        print('Model building using ' + init_weight_names[weight_init] + ' Initializer')
        history = model.fit(train_x, train_y, epochs=epochs, 
                           validation_split=0.2,
                           callbacks=[cb])
        print('Built model successfully.')
        print('')

In [ ]:
list_of_weights = [keras.initializers.RandomNormal(), keras.initializers.glorot_normal(), 
                    keras.initializers.he_normal(), keras.initializers.Orthogonal()]
init_weight_names = ['Random', 'Glorot', 'He', 'Orthogonal']
weight_initializer(list_of_weights, init_weight_names)

#### Open tensorboard in `weights` directory
Graphs will differ due to not setting seed.